<a href="https://colab.research.google.com/github/opsabarsec/NLP--film-genres-from-synopsis/blob/main/radix_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automatic assignment of genres from movie synopsis using supervised machine learning

## 1. Import libraries and load data

In [5]:
#packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# NLP libraries

from textblob import TextBlob, Word
import nltk
import re
import csv
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Deep learning libraries

from keras.models import Model
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, Input, LSTM, GlobalMaxPool1D, Dropout
from keras.preprocessing.text import Tokenizer

In [6]:
# load data
train= pd.read_csv('./data/train.csv')
test=pd.read_csv('./data/test.csv')

## 2. DATA PREPARATION 

In [7]:
nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
# function for text cleaning 
def preprocess_text(text):
    text = text.lower() # lowercase
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub(r"\'\n", " ", text) #line breaks
    #text = re.sub(r"\'\xa0", " ", text) # xa0 Unicode representing spaces
    #text = re.sub('\s+', ' ', text) # one or more whitespace characters
    text = text.strip(' ') # spaces
    # remove backslash-apostrophe 
    text = re.sub("\'", "", text) 
    # remove everything except alphabets 
    text = re.sub("[^a-zA-Z]"," ",text) 
    #lemmatize and remove stopwords
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    text = ' '.join(no_stopword_text) 
        
    return text

train['clean_plot'] = train['synopsis'].apply(lambda x: preprocess_text(x))
test['clean_plot'] = test['synopsis'].apply(lambda x: preprocess_text(x))

In [9]:
def lemma(text): # Lemmatization of cleaned body
        sent = TextBlob(text)
        tag_dict = {"J": 'a', 
                    "N": 'n', 
                    "V": 'v', 
                    "R": 'r'}
        words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in sent.tags]    
        lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
        seperator=' '
        lemma = seperator.join(lemmatized_list) 
        return lemma

In [10]:
train['lemma'] = train['clean_plot'].apply(lambda x: lemma(x))
test['lemma'] = test['clean_plot'].apply(lambda x: lemma(x))

## 3. Variables preparation 

In [11]:

X = train['lemma']
X_test = test['lemma']    

### 3.1 Target variable one hot encoding

In [12]:
#apply the onehot transformation for the genres vector
y = train['genres']
one_hot = MultiLabelBinarizer() # encoder for the  tags 
y_onehot = one_hot.fit_transform(y.str.split(' ')) 
y_bin = pd.DataFrame(y_onehot, columns=one_hot.classes_ ) # transform it to Pandas object

In [13]:
# tokenize
max_features = 5000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X))
list_tokenized_train = tokenizer.texts_to_sequences(X)
list_tokenized_test = tokenizer.texts_to_sequences(X_test)

In [14]:
#fix max comment lenght
maxlen = 100
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

## 4.The Model

In [15]:
#initialize parameters
inp = Input(shape=(maxlen, )) #maxlen defined earlier
embed_size = 128

In [16]:
# Neural network backbone
x = Embedding(max_features, embed_size)(inp)

x = LSTM(64, return_sequences=True,name='lstm_layer')(x)

x = GlobalMaxPool1D()(x)
x = Dropout(0.1)(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(len(y_bin.columns), activation="softmax")(x)

In [17]:
# build the model
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

In [18]:
# train the model
batch_size = 16
epochs = 3

hist = model.fit(X_t,y_onehot, batch_size=batch_size, epochs=epochs, validation_split=0.1)


Epoch 1/3
2055/2055 [==============================] - 88s 41ms/step - loss: 0.2701 - accuracy: 0.2916 - val_loss: 0.2375 - val_accuracy: 0.3727
Epoch 2/3
2055/2055 [==============================] - 82s 40ms/step - loss: 0.2229 - accuracy: 0.4109 - val_loss: 0.2099 - val_accuracy: 0.4515
Epoch 3/3
2055/2055 [==============================] - 80s 39ms/step - loss: 0.2000 - accuracy: 0.4817 - val_loss: 0.2045 - val_accuracy: 0.4806


## 5.The prediction

In [19]:
y_pred = model.predict(X_te, batch_size=batch_size, verbose=1)
print(y_pred.shape)

329/329 [==============================] - 5s 13ms/step
(5250, 19)


In [20]:
df_probs_all = pd.DataFrame(y_pred,columns=y_bin.columns)

df_probs_all.head()

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0.303940,0.170865,0.075160,0.029464,0.034091,0.004147,0.003006,0.027387,0.053119,1.176459e-04,0.021796,0.009377,0.001316,0.004451,0.005488,0.214212,0.032693,0.003265,0.006104
1,0.249033,0.075040,0.023714,0.008969,0.052143,0.012742,0.189909,0.157685,0.015774,2.169265e-04,0.003828,0.005781,0.006944,0.003986,0.007329,0.113913,0.040018,0.023611,0.009365
2,0.000371,0.000557,0.000362,0.000319,0.008084,0.000078,0.962359,0.022596,0.000258,4.416469e-07,0.000141,0.000038,0.002380,0.000092,0.000750,0.000193,0.000276,0.001140,0.000007
3,0.046851,0.076450,0.096233,0.047471,0.058890,0.001128,0.003898,0.023676,0.104039,5.013559e-05,0.337939,0.005439,0.001081,0.010169,0.005177,0.150504,0.030142,0.000398,0.000464
4,0.059372,0.003363,0.000421,0.000739,0.054022,0.278707,0.000960,0.215334,0.001670,1.190958e-02,0.020068,0.000109,0.000517,0.052654,0.011007,0.002513,0.282059,0.000651,0.003924


In [21]:
def top_5_predictions(df):
    N = 5
    cols = df.columns[:-1].tolist()
    a = df[cols].to_numpy().argsort()[:, :-N-1:-1]
    c = np.array(cols)[a]
    d = df[cols].to_numpy()[np.arange(a.shape[0])[:, None], a]
    df1 = pd.DataFrame(c).rename(columns=lambda x : f'max_{x+1}_col')

    predicted_genres = df1["max_1_col"] + ' ' + df1["max_2_col"]+ ' ' +df1["max_3_col"]+ ' ' + df1["max_4_col"]+ ' '+df1["max_5_col"]
    return predicted_genres

In [22]:
pred_gen = top_5_predictions(df_probs_all)

In [23]:
submission = pd.DataFrame(data= {'movie_id':test.movie_id,'predicted_genres':pred_gen})

In [24]:
submission.head()

,movie_id,predicted_genres
0,10827,Action Sci-Fi Adventure Animation Fantasy
1,51768,Action Documentary Drama Sci-Fi Adventure
2,36064,Documentary Drama Comedy Musical War
3,33763,Horror Sci-Fi Fantasy Animation Adventure
4,1146,Thriller Crime Drama Action Comedy


In [25]:
submission.to_csv('submission.csv',index=False)